### The Modelling Workhop 2020.12.22.##

Before we start modeling, it is necessary to know the basic parameters of the tools we will work with. Let Dalimil explain what is the difference between IDE and programing language.

As he speak, click on the dropdown menu "help" in the top bar and then on User Interface Tour. Enjoy the tour!

Note: Modeling with one "l" is mostly  the U.S. whereas modelling everywhere else.

For computational modeling, you should be familiar with basic python commands and functions.

In [ ]:
# import the basic lib


# import the most popular analytical lib with the correct alias

# set unconstrained tables
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 9999

# import the most popular plotting library (it will not be used here but its the basis in every session)


# choose the standart style
matplotlib.style.use('ggplot')
# set graph display in out IDE window 
%matplotlib inline

# for optional fun
import numpy as np

# show interactive output in cells - sometimes necessary for builder()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# import visualization widget



# import timing - again not used in this session but a handy tool anaway 
from time import sleep

# notice how kernel started working

In [ ]:
# test the fuctionality of imported libs
cobra.__version__

# test the version of pandas, matplotlib and numpy


In [ ]:
# importing the model
M = cobra.io.load_json_model(r"your_string")
# importing .mat model is just as easy. However, working with SBML is tricky and requires another lib (see documentation).

# let Dalimil explain how models are named

# note: SBML Pseudomonas model iJN1463 could not be imported due to error during id parsing

In [ ]:
# keep the original version of model unaltered - knock_out()
model = M.copy()
# ALWAYS ASSING THE MODEL TO VARIABLE MODEL!!

In [ ]:
# look at some basic stats
model

In [ ]:
# let Dalimil explain what dot notation is and then explore the two main components of S matrix:
# reactions and metaboltites
model.

Reactions

In [ ]:
# look at bounds name and reaction string
model.reactions..bounds
model.reactions..name
model.reactions..reaction

In [ ]:
# print separate bounds


Metabolites

In [ ]:
# look at name, compartment, charge and formula of glucose
model.metabolites.glu__D_c

In [ ]:
# print all reactions that produce/consume glucose
model.metabolites..reactions

Genes

In [ ]:
# there is a difference between GPR and actual genes
model.reactions..gene_reaction_rule # a boolean representation of required gene(s)
model.reactions..genes # a gene object

In [ ]:
# genes(loci) can also be accessed separately
model.genes.

Adding new metabolites and reactions


You were given the task of adding reactions to the model leading to the synthesis of polyhydroxybutyrate. Our goal is to show the ability of the model/bacterium to produce this compound.
These reactions are:

Ketothiolase PhaA
aacoa_c + coa_c ⇌ 2.0 accoa_c

Acetoacetyl-CoA reductase PhaB
aacoa_c + h_c + nadph_c ⇌ nadp_c + 3hbcoa__R_c

PHB synthetase PhaC
3hbcoa__R_c ⇌ coa_c + phb_c

First, it would be good to find their ID and check if the are not in the model already.
Next, do the same with metabolites.

What metabolites and reactions we need to add?

In [ ]:
# import before the task
from __future__ import print_function
from cobra import Model, Reaction, Metabolite

In [ ]:
# adding a metabolite
 = Metabolite(
'',
formula='',
name='',
compartment='')
# print the metabolite below


In [ ]:
# adding a reaction
 = Reaction("") #ID
.name = "" # full reaction name
.subsystem = "PHB synthesis" # e.g. Alternate Carbon Metabolism, nebo Central metabolism atd...
.lower_bound = . 
.upper_bound = . # you can also leave the default set.
# print the reaction below


In [ ]:
# adding reaction string
.add_metabolites({}) # metabolites are added as dict


# aacoa_c + h_c + nadph_c ⇌ nadp_c + 3hbcoa__R_c
# print the reaction below
.reaction

# reaction could be also build from string, but its dumb

In [ ]:
# add GPR
.gene_reaction_rule = "()"
.genes

In [ ]:
# verify that everything in the reaction is as it should be and save the reaction
model.add_reactions([])
# check the whole reaction again
model.reactions.

In [ ]:
# finally check the mass balance of the reactions you just added
.check_mass_balance()
# mass balanced reaction should return empty list
# discuss results with Dalimil

In [ ]:
# then save the model to a new file
cobra.io.save_json_model(model = model, filename = "") # don't forget to add the .json extension

Definition of a linear problem

In [ ]:
# look at the current objective function
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

In [ ]:
# set a new objective
model.objective = "PHBS_syn_1"

In [ ]:
# define the cultivation medium
model.medium
# what kind of medium is it?

In [ ]:
# do you see a carbon source there?
# set it on 10 mmol/gCDW/h

model.reactions.EX_glc__D_e.lower_bound = 
model.reactions.EX_glc__D_e.upper_bound = 

# OPTIMIZE!!
solution = model.optimize()
flux = solution.fluxes

solution
# is the result something that you would expect?
# consult with Dalimil (and with your newly gained knowledge) where the mistake occured

In [ ]:
# fix the problem here!


In [ ]:
# look at the fluxes of your reactions
model.reactions.KAT1.flux
model.reactions.AACOAR_syn.flux
model.reactions.PHBS_syn_1.flux

In [ ]:
# look at what is excreted or transported into cytoplasm
exchage_reactions = []
for reaction in flux.index:
    if "EX_" in reaction:
        exchage_reactions.append(reaction)
flux[exchage_reactions].sort_values(ascending = True)

In [ ]:
# you can also check individual compound(s)
model.metabolites..summary()

In [ ]:
# save the solution for later visualization - save as an .json object (Others are not permitted in escher)
import json
flux_dictionary = flux.to_dict()
with open('.json', 'w') as f:
    json.dump(flux_dictionary, f)

Simple glucose simulation

In [ ]:
# as you are already a semi-pro modeller try to simulate simple growth on glucose
# (resonable glucose uptake is 6 mmol/gCDW/h)
# save the solution
model.reactions.EX_glc__D_e.lower_bound = -6
model.reactions.EX_glc__D_e.upper_bound = 0

# OPTIMIZE!!
solution = model.optimize()
solution

In [ ]:
# look at fluxes through PHA synthesis reactions
list_noprod_rxn = []
for x in model.reactions:
    if x.products == []:
        list_noprod_rxn.append(x.id)

df = pd.DataFrame(list_noprod_rxn)
#df

PHA_rxn = []
for y in df[0]:
    if "PHA" in str(y):
        PHA_rxn.append(y)
    else:
        pass
#PHA_rxn


PHA_flux = []
for z in PHA_rxn:
    PHA_flux.append(model.reactions.get_by_id(z).flux)

df1 = pd.DataFrame([PHA_rxn, PHA_flux])
df1.transpose()

Simulation of reaction deletion

In [ ]:
# with the knowledge you already have try to simulate deletion of glucose dehydrogenase reaction (gcd)
# again save the solution

In [ ]:
# Now try to simulate MOMA of the same gcd deletion

In [ ]:
from cameo.flux_analysis.simulation import lmoma

In [ ]:


lmoma_result = lmoma(model, reference = solution.fluxes)

lmoma_result[model.reactions.BIOMASS_KT2440_WT3]

Visualization

In [ ]:
# its almost like a malování
Builder()
# for most purposes the web version is recommanded!
# https://escher.github.io/#/

### BONUS homework ###

In [ ]:
# this piece of code compared two different flux solutions are returns their differences (upregul, downregul, activated...)
index = 0
neg_reaction_id = []
neg_flux = []
pos_reaction_id = []
pos_flux = []
control_id = []
control_flux = []
no_control_flux = []
flux_here = []
control_neg_id = []
control_flux_neg = []
no_control_flux_neg = []
flux_here_neg = []
neg_flux_cont_id = []
neg_flux_cont = []
neg_flux_xyl_id = []
neg_flux_xyl = []
downregul_id = []
downregul_value = []
upregul_id = []
upregul_value = []
useless_reactions = []

# první je analyzovaný model = i
# druhý je srovnávaná kontrola = ii
for i, ii in zip(met_gluk, met_gluk):
    if i == 0 and ii == 0:
        pass
    elif i == ii:
        pass
    elif i > 0 and ii < 0:
        if i - ii > 0.5:
            pos_reaction_id.append(met_xyl_gluk.index.values[index])
            pos_flux.append(i - ii)
        else:
            pass
    elif i < 0 and ii > 0:
        if ii - i > 0.5:
            neg_reaction_id.append(met_xyl_gluk.index.values[index])
            neg_flux.append(ii-i)
        else:
            pass
    elif i == 0 and ii > 0:
        control_id.append(met_xyl_gluk.index.values[index])
        control_flux.append(ii)
    elif ii == 0 and i > 0:
        no_control_flux.append(met_xyl_gluk.index.values[index])
        flux_here.append(i)
    elif i == 0 and ii < 0:
        control_neg_id.append(met_xyl_gluk.index.values[index])
        control_flux_neg.append(ii)
    elif ii == 0 and i < 0:
        no_control_flux_neg.append(met_xyl_gluk.index.values[index])
        flux_here_neg.append(i)
    elif abs(ii) > abs(i) and ii / i >= 1.5:
        downregul_id.append(met_xyl_gluk.index.values[index])
        downregul_value.append(ii / i)
    elif abs(i) > abs(ii) and i / ii >= 1.5:
        upregul_id.append(met_xyl_gluk.index.values[index])
        upregul_value.append(i / ii)
    else:
        useless_reactions.append(met_xyl_gluk.index.values[index])  
                        
    index = index + 1

#useless_reactions
print("Reactions with positive flux in model but negative in control")    
name_pos_flux = []
for r in pos_reaction_id:
    var_name9 =  model.reactions.get_by_id(r).name
    name_pos_flux.append(var_name9)

pos_sum = pd.Series(data = pos_flux, index =  name_pos_flux, name = "fluxes difference")

pos_sum.to_frame()


print("Reactions with negative flux in model and positive in control")
name_neg_flux = []
for q in neg_reaction_id:
    var_name8 =  model.reactions.get_by_id(q).name
    name_neg_flux.append(var_name8)

neg_sum = pd.Series(data = neg_flux, index =  name_neg_flux, name = "fluxes difference")

neg_sum.to_frame()

print("No flux in model but X positive flux in control")
name_reactions_control = []
for b in control_id:
    var_name6 = model.reactions.get_by_id(b).name
    name_reactions_control.append(var_name6)

control = pd.Series(data = control_flux, index = name_reactions_control, name = "fluxes")

control.to_frame()

print("No flux in control but X positive flux in model")

name_reactions_no_control = []
for s in no_control_flux:
    var_name = model.reactions.get_by_id(s).name
    name_reactions_no_control.append(var_name)

no_control = pd.Series(data = flux_here, index = name_reactions_no_control, name = "fluxes")

no_control.to_frame()

print("No flux in model but X negative flux in control")

name_reactions_neg_control = []
for x in control_neg_id:
    var_name = model.reactions.get_by_id(x).name
    name_reactions_neg_control.append(var_name)


neg_control = pd.Series(data = control_flux_neg, index = name_reactions_neg_control, name = "fluxes")

neg_control.to_frame()

print("No flux in control but X negative flux in model")

name_reactions_neg_no_control = []
for y in no_control_flux_neg:
    var_name = model.reactions.get_by_id(y).name
    name_reactions_neg_no_control.append(var_name)

neg_no_control = pd.Series(data = flux_here_neg, index = name_reactions_neg_no_control, name = "fluxes")

neg_no_control.to_frame()

print("Reactions that are X fold downregulated")

name_reactions_downregul = []
for d in downregul_id:
    var_name1 = model.reactions.get_by_id(d).name
    name_reactions_downregul.append(var_name1)


downregul = pd.Series(data = downregul_value, index = name_reactions_downregul, name = "X fold downregulated")

downregul.to_frame()

print("Reactions that are X fold upregulated")

name_reactions_upregul = []
for o in upregul_id:
    var_name3 = model.reactions.get_by_id(o).name
    name_reactions_upregul.append(var_name3)

upregul = pd.Series(data = upregul_value, index = name_reactions_upregul, name = "X fold upregulated")

upregul.to_frame()